# Exporing aviation accidents data

Link to kagge: https://www.kaggle.com/khsamaha/aviation-accident-database-synopses

## Dataset purpose
The declared purpose for publishing the dataset was:
> * Which is the type accident often to happen? Which are the features relevant?
* What is season that there are more accident?
* The amateur have a influence on accident or injury severity?
* Do they take too long to make preliminary reports?
* What do scheme have more accident?
* Where are there more accident? - deprecated
* What do aircraft have more accident? -deprecated
* How do accidents evolve in the time of aviation in the United States?



In [1]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, preprocessing, svm
from sklearn.preprocessing import StandardScaler, Normalizer
import math
import re
import seaborn as sns


In [2]:
df = pd.read_csv('AviationDataEnd2016UP.csv', sep=',', header=0, encoding = 'iso-8859-1')

df.sample(10)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
10473,20100816X23350,Accident,WPR10LA406,2010-08-16,"Douglas, WY",United States,42.636111,-105.293611,NaN,NaN,...,Aerial Observation,NaN,NaN,2.0,NaN,NaN,VMC,MANEUVERING,Probable Cause,07/04/2011
14291,20080805X01159,Accident,LAX08LA240,2008-07-22,"Bountiful, UT",United States,40.869445,-111.926667,KBTF,Skypark Airport,...,Instructional,NaN,0.0,0.0,2.0,0.0,VMC,LANDING,Probable Cause,29/01/2009
70767,20001214X40107,Accident,NYC84LA204,1984-06-07,"BEDFORD, PA",United States,NaN,NaN,NaN,NaN,...,Other Work Use,NaN,0.0,0.0,0.0,1.0,VMC,MANEUVERING,Probable Cause,NaN
60182,20001213X32495,Accident,CHI88LA021A,1987-11-07,"DETROIT, MI",United States,NaN,NaN,DET,DETROIT CITY,...,Personal,NaN,0.0,0.0,0.0,5.0,VMC,STANDING,Probable Cause,24/02/1989
39832,20001208X05662,Accident,NYC96LA091,1996-04-25,"HOULTON, ME",United States,NaN,NaN,HUL,HOULTON INTERNATIONAL,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,TAKEOFF,Probable Cause,22/10/1996
16375,20070806X01111,Accident,NYC07LA166,2007-07-12,"Enterprise, AL",United States,31.299722,-85.899722,EDN,Enterprise Municipal Airport,...,Instructional,NaN,NaN,NaN,1.0,1.0,VMC,APPROACH,Probable Cause,06/05/2009
77807,20020917X03636,Accident,LAX82DVG26,1982-06-07,"WILLIAMS, CA",United States,NaN,NaN,NaN,NaN,...,Aerial Application,NaN,0.0,0.0,0.0,1.0,VMC,MANEUVERING,Probable Cause,07/06/1983
55910,20001213X28825,Accident,FTW89FA127,1989-07-03,"SANGER, TX",United States,NaN,NaN,NaN,BAR VK RANCH,...,Personal,NaN,2.0,0.0,0.0,0.0,VMC,TAKEOFF,Probable Cause,10/06/1991
15082,20080411X00457,Accident,CHI08CA091,2008-03-17,"Batavia, OH",United States,40.250000,-81.850000,NaN,NaN,...,Instructional,NaN,NaN,NaN,NaN,1.0,VMC,LANDING,Probable Cause,03/08/2009
11617,20100111X41106,Accident,ERA10LA111,2010-01-09,"Bayport, NY",United States,40.758611,-73.053611,23N,Bayport Aerodrome,...,Personal,NaN,NaN,NaN,NaN,2.0,VMC,NaN,Probable Cause,16/06/2010


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79293 entries, 0 to 79292
Data columns (total 31 columns):
Event.Id                  79293 non-null object
Investigation.Type        79293 non-null object
Accident.Number           79293 non-null object
Event.Date                79293 non-null object
Location                  79215 non-null object
Country                   78786 non-null object
Latitude                  25751 non-null float64
Longitude                 25742 non-null float64
Airport.Code              44666 non-null object
Airport.Name              47439 non-null object
Injury.Severity           79293 non-null object
Aircraft.Damage           76883 non-null object
Aircraft.Category         22477 non-null object
Registration.Number       76209 non-null object
Make                      79204 non-null object
Model                     79175 non-null object
Amateur.Built             78721 non-null object
Number.of.Engines         75175 non-null float64
Engine.Type             

In [4]:
df.describe()

/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Latitude,Longitude,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,25751.000000,25742.000000,75175.000000,55984.000000,53742.000000,54833.000000,66949.000000
mean,37.690421,-93.781061,1.148055,0.814679,0.317703,0.502581,5.790886
std,12.148019,39.243662,0.453847,6.233700,1.372924,2.781994,29.223016
min,-78.016945,-178.676111,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,89.218056,177.557778,18.000000,349.000000,111.000000,380.000000,699.000000


In [5]:
df.count()

Event.Id                  79293
Investigation.Type        79293
Accident.Number           79293
Event.Date                79293
Location                  79215
Country                   78786
Latitude                  25751
Longitude                 25742
Airport.Code              44666
Airport.Name              47439
Injury.Severity           79293
Aircraft.Damage           76883
Aircraft.Category         22477
Registration.Number       76209
Make                      79204
Model                     79175
Amateur.Built             78721
Number.of.Engines         75175
Engine.Type               75919
FAR.Description           22334
Schedule                  11501
Purpose.of.Flight         75399
Air.Carrier                3918
Total.Fatal.Injuries      55984
Total.Serious.Injuries    53742
Total.Minor.Injuries      54833
Total.Uninjured           66949
Weather.Condition         77136
Broad.Phase.of.Flight     73239
Report.Status             79293
Publication.Date          65819
dtype: i

In [6]:
# splitting date field in the components
import datetime

df['Year'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").year)
df['Month'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").month)
df['Day'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").day)


### Looking at some categories
I try to list some unique values in the categories fields to subsequently plot some data distribution over those.

In [7]:
categories = ['Investigation.Type',
             'Aircraft.Damage',
             'Aircraft.Category',
             'Amateur.Built',
             'Number.of.Engines',
             'Engine.Type',
             'FAR.Description',
             'Schedule',
             'Purpose.of.Flight',
             'Weather.Condition',
             'Broad.Phase.of.Flight',
             'Report.Status']

for c in categories:
    print(c , df[c].unique())



Investigation.Type ['Accident' 'Incident']
Aircraft.Damage ['Substantial' 'Destroyed' nan 'Minor']
Aircraft.Category ['Airplane' 'Helicopter' 'Weight-Shift' 'Glider' 'Unknown' 'Balloon'
 'Powered Parachute' 'Ultralight' 'Gyroplane' 'Gyrocraft' nan
 'Powered-Lift' 'Rocket' 'Blimp']
Amateur.Built ['Yes' 'No' nan]
Number.of.Engines [ nan   1.   2.   0.   4.   3.  18.]
Engine.Type ['Reciprocating' nan 'Turbo Prop' 'Turbo Fan' 'Turbo Shaft' 'Unknown'
 'Turbo Jet' 'Electric' 'REC, ELEC' 'None' 'TF, TJ' 'Hybrid Rocket'
 'REC, TJ, TJ' 'REC, TJ, REC, TJ' 'TJ, REC, REC, TJ']
FAR.Description ['Part 91: General Aviation' nan 'Part 135: Air Taxi & Commuter'
 'Public Aircraft' 'Part 121: Air Carrier' 'Unknown'
 'Non-U.S., Non-Commercial' 'Part 137: Agricultural' 'Non-U.S., Commercial'
 'Part 103: Ultralight' 'Part 133: Rotorcraft Ext. Load' 'Public Use'
 'Part 129: Foreign' 'Armed Forces' 'Part 437: Commercial Space Flight'
 'Part 91 Subpart K: Fractional' 'Part 125: 20+ Pax,6000+ lbs'
 'Part 91F: S

In [8]:
for c in categories:
    print('\n', df.groupby(by=c)[c].count().sort_values(ascending=False))
    print('Nulls: ', df[c].isnull().sum())



 Investigation.Type
Accident    76118
Incident     3175
Name: Investigation.Type, dtype: int64
Nulls:  0

 Aircraft.Damage
Substantial    57049
Destroyed      17322
Minor           2512
Name: Aircraft.Damage, dtype: int64
Nulls:  2410

 Aircraft.Category
Airplane             19273
Helicopter            2360
Glider                 381
Balloon                175
Gyrocraft              100
Weight-Shift            66
Powered Parachute       48
Unknown                 32
Ultralight              31
Powered-Lift             5
Blimp                    3
Gyroplane                2
Rocket                   1
Name: Aircraft.Category, dtype: int64
Nulls:  56816

 Amateur.Built
No     71105
Yes     7616
Name: Amateur.Built, dtype: int64
Nulls:  572

 Number.of.Engines
1.0     63082
2.0     10057
0.0      1143
3.0       477
4.0       415
18.0        1
Name: Number.of.Engines, dtype: int64
Nulls:  4118

 Engine.Type
Reciprocating       64598
Turbo Shaft          3305
Turbo Prop           3042
Turbo 

In [9]:
# null damages can't be defined
df[df['Aircraft.Damage'].isnull()]
df['Aircraft.Damage'].fillna('Unknown', inplace=True)

In [10]:
# null countries are outside US
df[df['Country'].isnull()]
df['Country'].fillna('Foreign', inplace=True)

In [11]:
# null categories can't be defined
df[df['Aircraft.Category'].isnull()]
df['Aircraft.Category'].fillna('Unknown', inplace=True)

#### Amateur producers
Instead of putting an 'unknown' value in the Amateur.Built field, I've collected all the producers and all the amateurs brands from the rest of the dataset and filled the null cells searching in the resulting two lists. For the remaining marks that are not present anywhere in the dataset I chose to set them as amateurs.

#### Number of engines
For the balloons I'll set this value to 0.
For the remaining, I'll make some assumptions and aproximations based on the producer type.
```
Number.of.Engines
1.0     63082
2.0     10057
0.0      1143
3.0       477
4.0       415
18.0        1
Name: Number.of.Engines, dtype: int64
Nulls:  4118
```

In [90]:
# Extracting producers and amateurs
producers = [x for x in df['Make'][df['Amateur.Built']== 'No'].unique() ]
amateurs  = [x for x in df['Make'][df['Amateur.Built']== 'Yes'].unique() ]


# Function that fixes the null in amateur.built
def fix_amateur_built(ab, m):
    if type(ab) == str:
        return ab
    else:
        if m in producers:
            return 'No'
        else:
            return 'Yes'
# Fix for Amateur.Built field        
df['Amateur.Built_notnull'] = df.apply(lambda x: fix_amateur_built(x['Amateur.Built'], x['Make']), axis=1)



# Function that fixes the null in number.of.engines
def fix_number_of_engines(noe, m):
    if noe >= 0:
        return noe
    else:
        if m in producers:
            return 1.5 # mean between a CESSNA and a BOEING...
        else:
            return 1 # 

# Fixing number 0 for balloons
df['Number.of.Engines'][df['Number.of.Engines'].isnull() & (df['Make'].str.contains('balloon', case=False))] = 0.0

df['Number.of.Engines_notnull'] = df.apply(lambda x: fix_number_of_engines(x['Number.of.Engines'], x['Make']), axis=1)


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
print(df.groupby(by='Amateur.Built')['Amateur.Built'].count().sort_values(ascending=False))
print('Nulls: ', df['Amateur.Built'].isnull().sum())

print(df.groupby(by='Amateur.Built_notnull')['Amateur.Built_notnull'].count().sort_values(ascending=False))
print('Nulls: ', df['Amateur.Built_notnull'].isnull().sum())

print(df.groupby(by='Number.of.Engines')['Number.of.Engines'].count().sort_values(ascending=False))
print('Nulls: ', df['Number.of.Engines'].isnull().sum())

print(df.groupby(by='Number.of.Engines_notnull')['Number.of.Engines_notnull'].count().sort_values(ascending=False))
print('Nulls: ', df['Number.of.Engines_notnull'].isnull().sum())


In [ ]:
# TODO --> CESSNAs, PIPERS and others have only one engine!!
df['Number.of.Engines'][df['Number.of.Engines'].isnull() & (df['Make'].str in producers)] = 2.0
df['Number.of.Engines'][df['Number.of.Engines'].isnull() & (df['Make'].str in amateurs)] = 1.0

print(df.groupby(by='Number.of.Engines')['Number.of.Engines'].count().sort_values(ascending=False))
print('Nulls: ', df['Number.of.Engines'].isnull().sum())

df[['Number.of.Engines', 'Make', 'Model']][df['Number.of.Engines'].isnull() ]

In [ ]:

les = {}

for l in categories:
    print("Labeling ", l)
    les[l] = preprocessing.LabelEncoder()
    les[l].fit(df[l])
    tr = les[l].transform(df[l]) 
    df.loc[:, l + '_feat'] = pd.Series(tr, index=df.index)
    print(les[l].classes_)

